In [65]:
import os
from dotenv import load_dotenv
load_dotenv()



True

In [16]:
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader('Hands_On_Machine_Learning_with_Scikit_Le.pdf')
loader

In [63]:
llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192",
    groq_api_key=os.getenv("GROQ_API_KEY")
)

In [17]:
document=loader.load()
document

[Document(metadata={'producer': 'calibre (4.8.0) [http://calibre-ebook.com]', 'creator': 'calibre (4.8.0) [http://calibre-ebook.com]', 'creationdate': '2020-02-11T18:06:47+00:00', 'author': 'Aurélien Géron', 'moddate': '2020-02-11T19:08:07+01:00', 'title': 'Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow', 'source': 'Hands_On_Machine_Learning_with_Scikit_Le.pdf', 'total_pages': 1150, 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'producer': 'calibre (4.8.0) [http://calibre-ebook.com]', 'creator': 'calibre (4.8.0) [http://calibre-ebook.com]', 'creationdate': '2020-02-11T18:06:47+00:00', 'author': 'Aurélien Géron', 'moddate': '2020-02-11T19:08:07+01:00', 'title': 'Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow', 'source': 'Hands_On_Machine_Learning_with_Scikit_Le.pdf', 'total_pages': 1150, 'page': 1, 'page_label': '2'}, page_content='Hands-On Machine Learning\nwith Scikit-Learn, Keras, and\nTensorFlow\nSECOND EDITION\nConcepts

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [19]:
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len)

In [20]:
chunks = text_splitter.split_documents(document)
chunks

[Document(metadata={'producer': 'calibre (4.8.0) [http://calibre-ebook.com]', 'creator': 'calibre (4.8.0) [http://calibre-ebook.com]', 'creationdate': '2020-02-11T18:06:47+00:00', 'author': 'Aurélien Géron', 'moddate': '2020-02-11T19:08:07+01:00', 'title': 'Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow', 'source': 'Hands_On_Machine_Learning_with_Scikit_Le.pdf', 'total_pages': 1150, 'page': 1, 'page_label': '2'}, page_content='Hands-On Machine Learning\nwith Scikit-Learn, Keras, and\nTensorFlow\nSECOND EDITION\nConcepts, Tools, and Techniques to Build Intelligent\nSystems\nAurélien Géron'),
 Document(metadata={'producer': 'calibre (4.8.0) [http://calibre-ebook.com]', 'creator': 'calibre (4.8.0) [http://calibre-ebook.com]', 'creationdate': '2020-02-11T18:06:47+00:00', 'author': 'Aurélien Géron', 'moddate': '2020-02-11T19:08:07+01:00', 'title': 'Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow', 'source': 'Hands_On_Machine_Learning_with_Scikit_Le.pdf

In [21]:
from langchain_community.embeddings import HuggingFaceEmbeddings

In [22]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

C:\Users\Vijay Kumar\AppData\Local\Temp\ipykernel_12224\619240441.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
e:\LANGCHAIN\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\LANGCHAIN\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not suppor

In [26]:
from langchain_community.vectorstores import FAISS

In [28]:
vector_store = FAISS.from_documents(chunks, embeddings)
vector_store

## Set Up Retrieval Chain

In [66]:
from langchain_core.prompts import ChatPromptTemplate
# Create prompt template
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context.
Think step by step before providing a detailed answer.
If you don't know the answer, simply say "I don't know" rather than making something up.

<context>
{context}
</context>

Question: {input}""")

In [67]:
from langchain.chains.combine_documents import create_stuff_documents_chain
# Create document chain
document_chain = create_stuff_documents_chain(llm, prompt)

In [68]:
from langchain.chains import create_retrieval_chain
# Create retriever
retriever = vector_store.as_retriever()

In [69]:

# Create retrieval chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [ ]:
 
response = retrieval_chain.invoke({"input": "what is machine learning"})
print("\nAnswer:", response["answer"], "\n")




Answer: According to the provided context, Machine Learning can be defined as:

* "the science (and art) of programming computers so they can learn from data."
* "the field of study that gives computers the ability to learn without being explicitly programmed." (Arthur Samuel, 1959)
* "A computer program is said to learn from experience E with respect to some task T and some performance measure P, if its performance on T, as measured by P, improves with experience E." (Tom Mitchell, 1997)

In simpler terms, Machine Learning is about building systems that can learn from data, which means getting better at some task, given some performance measure. 



In [80]:
response = retrieval_chain.invoke({"input": "what is gradient descent give me in detail"})
print("\nAnswer:", response["answer"], "\n")



Answer: Based on the provided context, here is a detailed answer to what Gradient Descent is:

**Gradient Descent is an iterative optimization algorithm** that finds the optimal solution to a problem by minimizing a cost function. In the context of machine learning, it is used to train a model by adjusting its parameters to best fit the training data.

**How it works:**

1. **Initialization**: The model parameters, θ, are initialized with random values.
2. **Iterative process**: The algorithm iteratively updates the model parameters to minimize the cost function (e.g., Mean Squared Error (MSE)).
3. **Gradient calculation**: At each iteration, the algorithm calculates the gradient of the cost function with respect to each model parameter θ. This gradient represents the rate of change of the cost function when the parameter θ is changed.
4. **Update parameters**: The algorithm updates the model parameters in the direction of the negative gradient, which is the direction of the steepest 

In [81]:
response = retrieval_chain.invoke({"input": "what is difference between logistic regression and Naieve bais algorithm?"})
print("\nAnswer:", response["answer"], "\n")



Answer: I don't know. The provided context does not mention Naive Bayes algorithm, so I cannot answer this question based on the given context. 



In [83]:
response = retrieval_chain.invoke({"input": "what is logistic regression? "})
print("\nAnswer:", response["answer"], "\n")


Answer: According to the provided context, Logistic Regression (also called Logit Regression) is a binary classifier that estimates the probability that an instance belongs to a particular class. If the estimated probability is greater than 50%, the model predicts that the instance belongs to that class (called the positive class, labeled “1”), and otherwise it predicts that it does not (i.e., it belongs to the negative class, labeled “0”). 



In [84]:
response = retrieval_chain.invoke({"input": "what is Linear regression and give me its equations also? "})
print("\nAnswer:", response["answer"], "\n")


Answer: Based on the provided context, Linear Regression is a model that makes a prediction by computing a weighted sum of the input features, plus a constant called the bias term (also called the intercept term).

The equation for Linear Regression model prediction is:

ˆy = θ0 + θ1x1 + θ2x2 + … + θnxn

where:

* ˆy is the prediction
* θ0 is the bias term (or intercept term)
* θ1, θ2, …, θn are the model's parameters
* x1, x2, …, xn are the input features

Alternatively, when θ and x are column vectors, the prediction can be represented as:

ˆy = θ⊺x

where θ⊺ is the transpose of θ (a row vector) and θ⊺x is the matrix multiplication of θ⊺ and x. 



In [85]:
response = retrieval_chain.invoke({"input": "what is k means clustering plot ELBOW curve? "})
print("\nAnswer:", response["answer"], "\n")


Answer: Based on the provided context, the elbow curve in K-Means clustering is a plot of the inertia (mean squared distance from each instance to its nearest centroid) as a function of the number of clusters (k). The curve has an "elbow" shape, where the inertia decreases rapidly at first, then slows down as k increases. The point at which the inertia stops dropping fast (the "elbow") is generally close to the optimal number of clusters. 

